In [ ]:
!pip install pydicom
!pip install scikit-learn
!pip install tqdm
!pip install scikit-learn
!pip install scikit-image
!pip install keras
!pip install tensorflow
!pip install pydot

In [1]:
import tensorflow

from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import *
from keras.layers.convolutional import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers.pooling import MaxPooling2D, GlobalMaxPool2D
from keras.layers.core import Lambda, RepeatVector, Reshape
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import tensorflow
import os
import numpy as np
import pydicom
from skimage.transform import resize
from tensorflow.keras.utils import img_to_array
from skimage import io
from sklearn.model_selection import train_test_split
from tqdm import *
import pydot
from keras.models import *



# Global variables

In [2]:
home_path = "D:/Datasets/chaos/"
image_width = 512
image_height = 512

# Load images

In [3]:
def get_pixels_hu(scan):
    
    image = scan.pixel_array
                      
    image = image.astype(np.int16)

    # Set outside-of-scan pixels to 1
    # The intercept is usually -1024, so air is approximately 0
    image[image == -2000] = 0
    
    # Convert to Hounsfield units (HU)
    intercept = scan.RescaleIntercept
    slope = scan.RescaleSlope
    
    if slope != 1:
        image = slope * image.astype(np.float64)
        image = image.astype(np.int16)
        
    image += np.int16(intercept)
    
    return np.array(image, dtype=np.int16)


def load_files_ids(path):
    return next(os.walk(path))[2]

def load_datasets(im_height, im_width , path):

    total = 0
    print("Looking in : ", path+'dicom')
    total = len(load_files_ids(path+'dicom'))
    print('Found : ', total)
    
    total = 40

    X = np.zeros((total, im_height, im_width, 1), dtype=np.float32)
    y = np.zeros((total, im_height, im_width, 1), dtype=np.float32)
    
    p = 0
    ids = load_files_ids(path+'dicom')
    ids = ids[:total]
    print("Loading No. images : ", len(ids))
    for n in trange(len(ids)):
        id_ = ids[n]
        # Load images
        ds = pydicom.dcmread(path+"dicom/"+id_)
        x_img = get_pixels_hu(ds)
        x_img = resize(x_img, (im_width, im_height, 1), mode = 'constant', preserve_range = True)
        # Load masks
        mask = img_to_array(io.imread(path+"ground/"+id_.replace(".dcm", ".png"), as_gray=True))
        mask = resize(mask, (im_width, im_height, 1), mode = 'constant', preserve_range = True)
        # Save images
        X[p] = x_img/255.0
        y[p] = mask/255.0
        p+=1
    
    return X, y

def load_data():
    X, y = load_datasets(image_width, image_height, home_path)
    return train_test_split(X, y, test_size=0.1, random_state=42)
X_train, X_valid, y_train, y_valid = load_data()

Looking in :  D:/Datasets/chaos/dicom
Found :  2874
Loading No. images :  40


100%|██████████| 40/40 [00:02<00:00, 16.13it/s]


# Create model

In [3]:
def convolution_block(block_input, num_filters=256, kernel_size=3, dilation_rate=1, padding="same", use_bias=False):
    x = Conv2D(
        num_filters,
        kernel_size=kernel_size,
        dilation_rate=dilation_rate,
        padding="same",
        use_bias=use_bias,
    )(block_input)
    x = BatchNormalization()(x)
    return (Activation('relu'))(x)


def DilatedSpatialPyramidPooling(dspp_input):
    dims = dspp_input.shape
    x = AveragePooling2D(pool_size=(dims[-3], dims[-2]))(dspp_input)
    x = convolution_block(x, kernel_size=1, use_bias=True)
    out_pool = UpSampling2D(
        size=(dims[-3] // x.shape[1], dims[-2] // x.shape[2]), interpolation="bilinear",
    )(x)

    out_1 = convolution_block(dspp_input, kernel_size=1, dilation_rate=4)
    out_6 = convolution_block(dspp_input, kernel_size=3, dilation_rate=8)
    out_12 = convolution_block(dspp_input, kernel_size=3, dilation_rate=12)
    out_18 = convolution_block(dspp_input, kernel_size=3, dilation_rate=16)

    x = Concatenate(axis=-1)([out_pool, out_1, out_6, out_12, out_18])
    output = convolution_block(x, kernel_size=1)
    return output

def ConvBox(input_filter, output_size, filters): 
    shape = (512/output_size[0], 512/output_size[1])
    pool = MaxPooling2D(pool_size=( int(shape[0]) , int(shape[1]) ))(input_filter)
    
    out_1 = convolution_block(pool, kernel_size=3, num_filters= filters)
    out_2 = convolution_block(out_1, kernel_size=3, num_filters= filters)
    out_3 = convolution_block(out_2, kernel_size=3, num_filters= filters)
    
    
   
    r = (Activation('sigmoid'))(out_3)
    
    return r
    
    

In [4]:

def Unet(nClasses, shape, patches_256, patches_128, patches_64, bounding_filter):
    dprate = 0.9
    kernel_size = (3,3)
        
    inputs = Input(shape)
    inputs_patche_256 = Input(patches_256)
    inputs_patche_128 = Input(patches_128)
    inputs_patche_64 = Input(patches_64)
    
    bb_inputs = Input(bounding_filter)        
    
    # =============== encode ===============
    
    # layer 1 : 512x512
    conv1 = Conv2D(64, kernel_size, padding='same')(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = (Activation('relu'))(conv1)
    conv1 = (Dropout(rate=dprate))(conv1)
    
    conv2 = Conv2D(64, kernel_size, padding='same')(conv1)
    conv2 = BatchNormalization()(conv2)
    conv2 = (Activation('relu'))(conv2)
    conv2 = (Dropout(rate=dprate))(conv2)

    
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    # residual connextion
    conv2 = Add()([conv1,conv2])
    
    # bounding box filter 1
    filter_1 = ConvBox(bb_inputs, (512, 512), 64)
    conv2 =  Multiply()([conv2, filter_1])
    
    # +++++++++++++++ First patche level ++++++++++++++++++
    conv_p1 = Conv2D(64, kernel_size, padding='same')(inputs_patche_256)
    patches_1 = Concatenate()([pool1, conv_p1])
    
    
    # layer 2 : 256x256
    conv3 = Conv2D(128, kernel_size, padding='same')(patches_1)
    conv3 = BatchNormalization()(conv3)
    conv3 = (Activation('relu'))(conv3)
    conv3 = (Dropout(rate=dprate))(conv3)
    
    conv4 = Conv2D(128, kernel_size, padding='same')(conv3)
    conv4 = BatchNormalization()(conv4)
    conv4 = (Activation('relu'))(conv4)
    conv4 = (Dropout(rate=dprate))(conv4)
    
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv4)
    
    # residual connextion
    conv4 = Add()([conv3,conv4])
    
    # bounding box filter 2
    filter_2 = ConvBox(bb_inputs, (256, 256), 128)
    conv4 =  Multiply()([conv4, filter_2])
    
    #  +++++++++++++++ Second patche level ++++++++++++++++++
    conv_p2 = Conv2D(128, kernel_size, padding='same')(inputs_patche_128)
    patches_2 = Concatenate()([pool2, conv_p2])
    
    # layer 3 : 128x128
    conv5 = Conv2D(256, kernel_size, padding='same')(patches_2)
    conv5 = BatchNormalization()(conv5)
    conv5 = (Activation('relu'))(conv5)
    conv5 = (Dropout(rate=dprate))(conv5)
    
    conv6 = Conv2D(256, kernel_size, padding='same')(conv5)
    conv6 = BatchNormalization()(conv6)
    conv6 = (Activation('relu'))(conv6)
    conv6 = (Dropout(rate=dprate))(conv6)
    
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv6)
    
    # residual connextion
    conv6 = Add()([conv5,conv6])
    
    # bounding box filter 2
    filter_3 = ConvBox(bb_inputs, (128, 128), 256)
    conv6 =  Multiply()([conv6, filter_3])
    
    #  +++++++++++++++ Third patche level ++++++++++++++++++
    conv_p3 = Conv2D(128, kernel_size, padding='same')(inputs_patche_64)
    patches_3 = Concatenate()([pool3, conv_p3])
    
    # layer 4 : 64x64
    conv7 = Conv2D(512, kernel_size, padding='same')(patches_3)
    conv7 = BatchNormalization()(conv7)
    conv7 = (Activation('relu'))(conv7)
    conv7 = (Dropout(rate=dprate))(conv7)
    
    conv8 = Conv2D(512, kernel_size, padding='same')(conv7)
    conv8 = BatchNormalization()(conv8)
    conv8 = (Activation('relu'))(conv8)
    conv8 = (Dropout(rate=dprate))(conv8)
    
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv8)

    # layer 5 : 32x32
    
    conv9 = Conv2D(1024, kernel_size, padding='same')(pool4)
    conv9 = BatchNormalization()(conv9)
    
    # ASPP module
    conv9 = DilatedSpatialPyramidPooling(conv9)
    
    # =============== decode ===============
    
    o = (UpSampling2D((2, 2)))(conv9)
    # skip connexion 64
    o = (concatenate([o, conv8], axis=-1))
    o = (Conv2D(512, kernel_size, padding='same'))(o)
    o = (BatchNormalization())(o)
    o = (Activation('relu'))(o)
    o = (Dropout(rate=dprate))(o)
    
    outputs_64 = Activation('softmax')(o)
    
    o = (UpSampling2D((2, 2)))(o)
    # skip connexion 128
    o = (concatenate([o, conv6], axis=-1))
    o = (Conv2D(256, kernel_size, padding='same'))(o)
    o = (BatchNormalization())(o)
    o = (Activation('relu'))(o)
    o = (Dropout(rate=dprate))(o)
    
    outputs_128 = Activation('softmax')(o)
    
    o = (UpSampling2D((2, 2)))(o)
    # skip connexion 256
    o = (concatenate([o, conv4], axis=-1))
    o = (Conv2D(128, kernel_size, padding='same'))(o)
    o = (BatchNormalization())(o)
    o = (Activation('relu'))(o)
    o = (Dropout(rate=dprate))(o)
    
    outputs_256 = Activation('softmax')(o)

    o = (UpSampling2D((2, 2)))(o)
    # skip connexion 512
    o = (concatenate([o, conv2], axis=-1))
    o = (Conv2D(64, kernel_size, padding='same'))(o)
    o = (BatchNormalization())(o)
    o = (Activation('relu'))(o)
    o = (Dropout(rate=dprate))(o)
    
    o = (Conv2D(64, kernel_size, padding='same'))(o)
    o = (BatchNormalization())(o)
    o = (Activation('relu'))(o)
    o = (Dropout(rate=dprate))(o)
    

    o = Conv2D(nClasses, (1,1), padding='same')(o)
    outputs_512 = Activation('softmax')(o)

    model = Model(inputs=[inputs, inputs_patche_256, inputs_patche_128, inputs_patche_64, bb_inputs], 
                  outputs=[outputs_512, outputs_256, outputs_128, outputs_64, outputs_128 ])
    return model

In [5]:
unet_model = Unet(1, (512, 512, 1), (256, 256, 1), (128, 128, 1), (64, 64, 1), (512, 512, 1))

from keras.losses import binary_crossentropy

def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

def my_loss(i, j):
    def custom_loss(y_true, y_pred):
        return i*dice_coef_loss(y_true, y_pred) + j*binary_crossentropy(y_true, y_pred)
    return custom_loss


callbacks = [
    EarlyStopping(patience=10, verbose=1),
    ReduceLROnPlateau(factor=0.1, patience=5, min_lr=0.00001, verbose=1),
    ModelCheckpoint('unet-0.h5', verbose=1, save_best_only=True, save_weights_only=True)
]

losses = {
    "loss1": my_loss(4, 1),
    "loss2": my_loss(3, 2),
    "loss3": my_loss(2, 3),
    "loss4": my_loss(1, 4),
}
lossWeights = {"loss1": 1.0, "loss2": 1.0, "loss3": 1.0, "loss4": 1.0}

unet_model.compile(optimizer=Adam(), loss=losses, loss_weights=lossWeights,  metrics=[tensorflow.keras.metrics.Accuracy(), tensorflow.keras.metrics.MeanIoU(num_classes=2)])


In [7]:
unet_model.summary(line_length=220,positions=[.2, .3, .4, 1.])

Model: "model"
____________________________________________________________________________________________________________________________________________________________________________________________________________________________
 Layer (type)                               Output Shape          Param #               Connected to                                                                                                                        
 input_1 (InputLayer)                       [(None, 512, 512, 1)  0                     []                                                                                                                                  
                                            ]                                                                                                                                                                               
                                                                                                     

In [ ]:
plot_model(unet_model, to_file='unet.png', show_shapes=True, show_layer_names=False)

In [ ]:
results = unet_model.fit(X_train, y_train, batch_size=1, epochs=5, callbacks=callbacks, validation_data=(X_valid, y_valid))

In [ ]:
plt.figure(figsize=(8, 8))
plt.title("Learning curve")
plt.plot(results.history["loss"], label="loss")
plt.plot(results.history["val_loss"], label="val_loss")
plt.plot( np.argmin(results.history["val_loss"]), np.min(results.history["val_loss"]), marker="x", color="r", label="best model")
plt.xlabel("Epochs")
plt.ylabel("log_loss")
plt.legend()
plt.savefig('Learning_curve_loss.png')

In [ ]:
plt.figure(figsize=(8, 8))
plt.title("Learning curve")
plt.plot(results.history["accuracy"], label="accuracy")
plt.plot(results.history["val_accuracy"], label="val_accuracy")
plt.plot( np.argmin(results.history["val_accuracy"]), np.max(results.history["val_accuracy"]), marker="x", color="r", label="best model")
plt.xlabel("Epochs")
plt.ylabel("log_accuracy")
plt.legend();
plt.savefig('Learning_curve.png')

In [ ]:
plt.figure(figsize=(8, 8))
plt.title("Metrics")
plt.plot(results.history["dice_coef"], label="dice_coef")
plt.plot(results.history["dice_coef_loss"], label="dice_coef_loss")
plt.plot(results.history["jaccard_index"], label="jaccard_index")
plt.xlabel("Epochs")
plt.ylabel("log_acc")
plt.legend();
plt.savefig('metrics.png')

In [ ]:
import PIL
import matplotlib.pyplot as plt

gray_img = img_to_array(io.imread("test.png", as_gray=True))
#gray_img = resize(gray_img, (512, 512, 1), mode = 'constant', preserve_range = True)
gray_img = np.expand_dims(gray_img, axis=0)
print(gray_img.shape)
plt.imshow(gray_img[0], cmap='gray')